In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [23]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

### Create DataFrames to match tables

In [4]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [40]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)","customer_count")
customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   42596797|             1|
|   12296525|             1|
|   49101505|             1|
|   28377689|             1|
|   14468845|             1|
+-----------+--------------+
only showing top 5 rows



In [39]:
# Create the products_table DataFrame and drop duplicates. 
products_df = (df.select(["product_id","product_title"])).drop_duplicates()
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B005GUQCGK|Everest Gym Bag w...|
|B00M6O0WQK|Sprayground Unise...|
|B00UFFW5XK|Polare Italian Le...|
|B00J8O730E|Nicole Lee Shoppi...|
|B0028Y67PI|David King & Co. ...|
+----------+--------------------+
only showing top 5 rows



In [38]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(['review_id','customer_id','product_id','product_parent',(to_date("review_date", 'yyyy-MM-dd').alias("review_date"))])
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R9CO86UUJCAW5|   40884699|B00VGTN02Y|     786681372| 2015-08-31|
|R3PR8X6QGVJ8B1|   23208852|B005KIWL0E|     618251799| 2015-08-31|
|R39BO2819ABUPF|   17100246|B007UNSHJ6|     810480328| 2015-08-31|
|R3ADL7V6EGGEEP|   13533670|B00WHFNXB4|      20765193| 2015-08-31|
|R1OXYPBPLVRMI5|   38541771|B000M5NBYU|     812890709| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [37]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R9CO86UUJCAW5|          3|            0|          0|   N|                Y|
|R3PR8X6QGVJ8B1|          5|            0|          0|   N|                Y|
|R39BO2819ABUPF|          4|            0|          0|   N|                Y|
|R3ADL7V6EGGEEP|          4|            0|          0|   N|                Y|
|R1OXYPBPLVRMI5|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [41]:
from pyspark.sql.functions import col

bias_df = vine_df.filter(col("total_votes") >= 20)
bias_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [42]:
bias_df = bias_df.filter(col("helpful_votes") / col("total_votes") >= 0.5)
bias_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [32]:
vine_y_df = bias_df.filter(col('vine') == 'Y')

In [33]:
vine_n_df = bias_df.filter(col('vine') == 'N')

In [34]:
# total number of 5 star reviews where total votes > 20 and helpful_votes / total votes > 0.5
ttl_five_star = bias_df[bias_df['star_rating'] == 5].count()
ttl_all_star = bias_df.count()
print('The total number of all star reviews:{}'.format(ttl_all_star))
print('The total number of five star reviews:{}'.format(ttl_five_star))
print('The percentage of five star reviews:{:.2f}%'.format((ttl_five_star/ttl_all_star)*100))

The total number of all star reviews:6711
The total number of five star reviews:3458
The percentage of five star reviews:51.53%


In [35]:
# % of five star reviews for Vine program (unpaid), vine == 'N'.
no_five_star = vine_n_df[vine_n_df['star_rating'] == 5].count()
no_reviews = vine_n_df.count()
print('The total number of all star reviews (not vine):{}'.format(no_reviews))
print('The total number of five star reviews (not vine):{}'.format(no_five_star))
print('The percentage of five star reviews:{:.2f}%'.format((no_five_star/no_reviews)*100))

The total number of all star reviews (not vine):6690
The total number of five star reviews (not vine):3448
The percentage of five star reviews:51.54%


In [36]:
# % of five star reviews for Vine program (paid), vine == 'Y'.
yes_five_star = vine_y_df[vine_y_df['star_rating'] == 5].count()
yes_reviews = vine_y_df.count()
print('The total number of all star reviews (vine):{}'.format(yes_reviews))
print('The total number of five star reviews (vine):{}'.format(yes_five_star))
print('The percentage of five star reviews:{:.2f}%'.format((yes_five_star / yes_reviews) * 100))


The total number of all star reviews (vine):21
The total number of five star reviews (vine):10
The percentage of five star reviews:47.62%


### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Store environmental variable
from getpass import getpass
password = getpass('XXXXXXX')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cx3jpmtuzudq.us-west-1.rds.amazonaws.com:5432/aws_review_data"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [29]:
#df= df.groupBy("product_id","product_title").count().filter("count > 1")
#df.drop('count').show()

In [27]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [28]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)